# Multivariate Fuzzy C-means method: Implementation

## Equations

### $J= \sum_{i=1}^{c} \sum_{k=1}^{n} \sum_{j=1}^{p} \left(u_{ijk} \right)^{m} d_{ijk}$ - Objective function to minimize.

### $d_{ijk} = \left(x_{jk} - y_{ij} \right)^{2}$ - euclidian distance squared.

### $ y_{ij} = \frac{\sum_{k=1}^{n} \left(u_{ijk} \right)^{m} x_{jk}} {\sum_{k=1}^{n} \left(u_{ijk}\right)^{m}}$ - prototype coordinate of a given cluster in feature j.

### $ u_{ijk} =  \left[\sum_{h=1}^{c}\sum_{l=1}^{p} \left(\frac{d_{ijk}}{d_{hlk}}\right)^{(1/(m-1))}  \right]^{-1} $ - membership degree of pattern k in cluster $C_{i}$ on the feature j.

### $\delta_{ik} = \sum_{j=1}^{p} u_{ijk}$ - represents an aggregation measure for all the p features.

## Constraints:

### - $u_{ijk} \in [0, 1]$ for all i, j and k;
### - $0 < \sum_{j=1}^{p} \sum_{k=1}^{n} u_{ijk} < n$ for all i and
### - $\sum_{i=1}^{c}\sum_{j=1}^{p}u_{ijk} = 1$ for all k.

## Importando bibliotecas

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

## Tratamento dos dados

In [2]:
def gerar_configuracao(mu_list, sigma_list, tamanhos, config_id):
    dfs = []
    for i, (mu, sigma2, n) in enumerate(zip(mu_list, sigma_list, tamanhos)):
        Sigma = np.diag(sigma2)
        data = np.random.multivariate_normal(mu, Sigma, n)
        df = pd.DataFrame(data, columns=["x1", "x2"])
        df["class"] = i + 1
        dfs.append(df)
    df_config = pd.concat(dfs, ignore_index=True)
    df_config["config"] = config_id
    return df_config

np.random.seed(42)  # reprodutibilidade

# -------------------------------
# Configuração 1
mu_1 = [[5, 0], [15, 5], [18, 14]]
sigma2_1 = [[81, 9], [9, 100], [25, 36]]
n1 = [200, 100, 50]
df1 = gerar_configuracao(mu_1, sigma2_1, n1, config_id=1)
l1 = "Classes elípticas de tamanhos diferentes"

# -------------------------------
# Configuração 2
mu_2 = [[0, 0], [30, 0], [12, 25]]
sigma2_2 = [[100, 100], [49, 49], [16, 16]]
n2 = [200, 100, 50]
df2 = gerar_configuracao(mu_2, sigma2_2, n2, config_id=2)
l2 = "Classes esféricas de tamanhos diferentes"

# -------------------------------
# Configuração 3
mu_3 = [[0, 0], [15, 5], [15, -5]]
sigma2_3 = [[100, 4], [100, 4], [100, 4]]
n3 = [100, 100, 100]
df3 = gerar_configuracao(mu_3, sigma2_3, n3, config_id=3)
l3 = "Classes elípticas de tamanhos iguais"

# -------------------------------
# Configuração 4
mu_4 = [[0, 0], [15, 0], [-15, 0]]
sigma2_4 = [[16, 16], [16, 16], [16, 16]]
n4 = [100, 100, 100]
df4 = gerar_configuracao(mu_4, sigma2_4, n4, config_id=4)
l4 = "Classes elípticas de tamanhos iguais"

# -------------------------------
# Configuração 5
mu_5 = [[5, 0], [15, 5], [10, -7], [3, 15]]
sigma2_5 = [[81, 9], [9, 100], [49, 16], [25, 25]]
n5 = [50, 50, 50, 50]
df5 = gerar_configuracao(mu_5, sigma2_5, n5, config_id=5)
l5 = "3 classes elípticas e 1 esférica"

# -------------------------------
# Configuração 6
mu_6 = [[5, 0], [15, 5], [12, -12], [7, 17]]
sigma2_6 = [[81, 9], [9, 100], [16, 16], [25, 25]]
n6 = [50, 50, 50, 50]
df6 = gerar_configuracao(mu_6, sigma2_6, n6, config_id=6)
l6 = "2 classes elípticas e 2 esféricas"

# -------------------------------
# Configuração 7
mu_7 = [[0, 0], [18, 0], [-18, 0], [0, -12]]
sigma2_7 = [[12, 12], [20, 20], [16, 16], [81, 20]]
n7 = [50, 50, 50, 50]
df7 = gerar_configuracao(mu_7, sigma2_7, n7, config_id=7)
l7 = "1 classe elíptica e 3 esféricas"

In [3]:
class MFCM():
    def __init__(self, c, X, m):
        self.c = c
        self.n = X.shape[0]
        self.p = X.shape[1]
        self.m = m

        ##
        self.global_var = np.var(X, axis=0).mean() # pré-calcula a variância global dos dados para regularização

    def initialize_u(self):
        u_flat = np.random.dirichlet(alpha=np.ones(self.c * self.p), size=self.n)
        return u_flat.reshape(self.n, self.c, self.p)
    
    def initialize_lambda(self):
        return np.ones((self.c, self.p))
    
    def find_centroides(self, X, U):
        u_m = U ** self.m
        numerador = np.sum(u_m * X[:, np.newaxis, :], axis=0)
        denominador = np.sum(u_m, axis=0)
        denominador = np.fmax(denominador, np.finfo(np.float64).eps)
        return numerador / denominador
    
    def get_distances(self, X, V):
        return (X[:, np.newaxis, :] - V[np.newaxis, :, :]) ** 2

    def update_u(self, D, Lambda):
        power = 1.0 / (self.m - 1)
        eps = np.finfo(np.float64).eps
        
        weighted_dist = D * Lambda
        weighted_dist = np.fmax(weighted_dist, eps) 
        
        term = weighted_dist ** (-power)
        
        denominator = np.sum(term, axis=(1, 2), keepdims=True)
        denominator = np.fmax(denominator, eps)
        
        return term / denominator
    
    def update_lambda(self, D, U):
        eps = np.finfo(np.float64).eps
        
        term_k = (U ** self.m) * np.fmax(D, eps)
        S_ij = np.sum(term_k, axis=0) 
        
        # Regularização Aditiva
        ##
        regularization_factor = 0.01 * self.global_var
        S_ij = S_ij + regularization_factor 
        
        prod_S = np.prod(S_ij, axis=1, keepdims=True)
        numerator = prod_S ** (1.0 / self.p)
        
        Lambda = numerator / S_ij
        
        return Lambda

    ##
    def calculate_objective_function(self, D, U, Lambda):
        term = (U ** self.m) * D
        sum_k = np.sum(term, axis=0)
        return np.sum(Lambda * sum_k)
    

def mfcm_run(dados, num_clusters, m=2, max_iter=1000, epsilon=1e-6):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(dados)
    
    mfcm = MFCM(c=num_clusters, X=X_scaled, m=m)
    
    indices = np.random.choice(X_scaled.shape[0], num_clusters, replace=False)
    centroids = X_scaled[indices]
    
    Lambda = np.ones((num_clusters, X_scaled.shape[1]))
    
    D = mfcm.get_distances(X_scaled, centroids)
    U = mfcm.update_u(D, Lambda)
    
    WARM_UP_ITERS = 20 

    for i in range(max_iter):
        U_old = U.copy()
        
        centroids = mfcm.find_centroides(X_scaled, U)
        D = mfcm.get_distances(X_scaled, centroids)
        
        ##
        if i >= WARM_UP_ITERS:
            Lambda = mfcm.update_lambda(D, U) 
        
        U = mfcm.update_u(D, Lambda)
        
        if np.linalg.norm(U - U_old) < epsilon:
            break
        
    J_final = mfcm.calculate_objective_function(D, U, Lambda)
    Delta = np.sum(U, axis=2)
    
    return centroids, U, Delta, Lambda, J_final

def calculate_objective_function(self, D, U, Lambda):
        term = (U ** self.m) * D
        sum_k = np.sum(term, axis=0)
        weighted_sum = Lambda * sum_k
        J = np.sum(weighted_sum)
        return J


def run_final_experiment(dados, labels, num_clusters=3, num_trials=100, restarts=50):
    results_ari = []
    results_ami = []
    
    for t in range(num_trials):
        best_J = np.inf
        best_pred = None
        
        for r in range(restarts):
            try:
                centroids, U, Delta, Lambda, J = mfcm_run(dados, num_clusters, m=2)
                
                if J > 1e-3 and J < best_J:
                    best_J = J
                    best_pred = np.argmax(Delta, axis=1)
            except:
                continue
        
        if best_pred is not None:
            ari = adjusted_rand_score(labels, best_pred)
            ami = adjusted_mutual_info_score(labels, best_pred)
            
            results_ari.append(ari)
            results_ami.append(ami)
            
            print(f"Trial {t+1}: J={best_J:.4f} | ARI={ari:.4f} | AMI={ami:.4f}")
        else:
            print(f"Trial {t+1}: Falha (Singularidade em todas as tentativas).")

    print(f"Mean ARI: {np.mean(results_ari):.4f} +/- {np.std(results_ari):.4f}")
    print(f"Mean AMI: {np.mean(results_ami):.4f} +/- {np.std(results_ami):.4f}")

In [4]:
np.random.seed(42)

i = 1
for df in [df1, df2, df3, df4, df5, df6, df7]:
    print(f"\nConfiguração {i}")
    if i == 5 or i == 6 or i == 7:
        num_clusters = 4
    else:
        num_clusters = 3
    df.drop("config", axis=1, inplace=True)
    if i == 5 or i == 6 or i == 7:
        df["class"].replace({1: 0, 2: 1, 3: 2, 4: 3}, inplace=True)
    else: 
        df["class"].replace({1: 0, 2: 1, 3: 2}, inplace=True)
    labels = df["class"].values
    df.drop("class", axis=1, inplace=True)
    dados = df.to_numpy()
    run_final_experiment(dados, labels, num_clusters=num_clusters, num_trials=100, restarts=50)
    i += 1


Configuração 1
Trial 1: J=0.7571 | ARI=0.2580 | AMI=0.2785
Trial 2: J=0.7571 | ARI=0.2794 | AMI=0.3199
Trial 3: J=0.7607 | ARI=0.2223 | AMI=0.2540
Trial 4: J=0.7553 | ARI=0.4693 | AMI=0.3260
Trial 5: J=0.7496 | ARI=0.4761 | AMI=0.3363
Trial 6: J=0.7570 | ARI=0.2636 | AMI=0.2962
Trial 7: J=0.7544 | ARI=0.2613 | AMI=0.2805
Trial 8: J=0.7509 | ARI=0.4612 | AMI=0.3250
Trial 9: J=0.7546 | ARI=0.4715 | AMI=0.3235
Trial 10: J=0.7573 | ARI=0.2799 | AMI=0.3195
Trial 11: J=0.7600 | ARI=0.2601 | AMI=0.2891
Trial 12: J=0.7600 | ARI=0.2532 | AMI=0.2693
Trial 13: J=0.7538 | ARI=0.4800 | AMI=0.3316
Trial 14: J=0.7546 | ARI=0.4715 | AMI=0.3235
Trial 15: J=0.7558 | ARI=0.4629 | AMI=0.3159
Trial 16: J=0.7494 | ARI=0.4680 | AMI=0.3280
Trial 17: J=0.7539 | ARI=0.2677 | AMI=0.2817
Trial 18: J=0.7521 | ARI=0.2618 | AMI=0.2757
Trial 19: J=0.7585 | ARI=0.4187 | AMI=0.2869
Trial 20: J=0.7546 | ARI=0.4715 | AMI=0.3235
Trial 21: J=0.7571 | ARI=0.2580 | AMI=0.2785
Trial 22: J=0.7505 | ARI=0.2668 | AMI=0.2856
Tri